In [1]:
!pip install transformers datasets

In [ ]:

!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-geometric
!pip install numpy


Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, DataLoader

# 1. Load your pickled dataset
df_train = pd.read_pickle('/content/drive/MyDrive/MELD/train_with_emb.pkl')
df_dev = pd.read_pickle('/content/drive/MyDrive/MELD/dev_with_emb.pkl')
df_test = pd.read_pickle('/content/drive/MyDrive/MELD/test_with_emb.pkl')

# 2. Define a sentiment → ID helper
_SENT2ID = {'negative': 0, 'neutral': 1, 'positive': 2}
def map_sentiments_to_ids(sentiments):
    return [_SENT2ID[s] for s in sentiments]

# 3. Build edges + sentiment‐shift attributes
def compute_sentiment_edge_features(sent_ids):
    src, dst, attr = [], [], []
    L = len(sent_ids)
    for i in range(L):
        if i > 0:
            src.append(i); dst.append(i-1)
            attr.append([sent_ids[i], sent_ids[i-1]])
        if i < L-1:
            src.append(i); dst.append(i+1)
            attr.append([sent_ids[i], sent_ids[i+1]])
    # self-loops
    for i in range(L):
        src.append(i); dst.append(i)
        attr.append([sent_ids[i], sent_ids[i]])
    edge_index = torch.tensor([src, dst], dtype=torch.long)
    edge_attr  = torch.tensor(attr, dtype=torch.long)
    return edge_index, edge_attr

# 4. Function to build one graph
def build_linecongat_graph(emb_list, sentiments, emotion_ids=None):
    """
    emb_list     : list of np.ndarray, each shape (768,) for one utterance
    sentiments   : list of strings of length L, each 'negative'/'neutral'/'positive'
    emotion_ids  : optional list of int labels length L

    Returns a torch_geometric.data.Data with:
      - x                : [L, 768] node features
      - edge_index       : [2, E] directed edges (i→i-1, i→i+1, i→i)
      - edge_attr        : [E, 2] sentiment IDs [s_i, s_j]
      - sentiment_shift  : [E] = s_j – s_i for each edge
      - y (if provided)  : [L] node‐level emotion labels
    """
    x = torch.from_numpy(np.stack(emb_list)).float()  # [L, 768]
    sent_ids = map_sentiments_to_ids(sentiments)
    edge_index, edge_attr = compute_sentiment_edge_features(sent_ids)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    data.sentiment_shift = data.edge_attr[:, 1] - data.edge_attr[:, 0]
    if emotion_ids is not None:
        data.y = torch.tensor(emotion_ids, dtype=torch.long)
    return data

# 5. Define an emotion → index map for labels
label2idx = {
    'neutral': 0, 'surprise': 1, 'joy': 2, 'sadness': 3,
    'anger': 4, 'fear': 5, 'disgust': 6, 'other': 7
}

# 6. Iterate over conversations
graphs_train = []
for dlg_id, grp in df_train.groupby('dialogue_id', sort=True):
    grp = grp.sort_values('utterance_id')
    emb_list   = grp['emb_array'].tolist()
    sentiments = grp['sentiment_label'].tolist()
    emos_int   = [label2idx[e] for e in grp['label'].tolist()]
    g = build_linecongat_graph(emb_list, sentiments, emos_int)
    graphs_train.append(g)

# 7. (Optional) wrap in a DataLoader for batching
train_loader = DataLoader(graphs_train, batch_size=16, shuffle=True)

print(f"Built {len(graphs_train)} graphs – example graph has {graphs_train[0].num_nodes} nodes.")

graphs_dev = []
for dlg_id, grp in df_dev.groupby('dialogue_id', sort=True):
    grp = grp.sort_values('utterance_id')
    emb_list   = grp['emb_array'].tolist()
    sentiments = grp['sentiment_label'].tolist()
    emos_int   = [label2idx[e] for e in grp['label'].tolist()]
    g_dev = build_linecongat_graph(emb_list, sentiments, emos_int)
    graphs_dev.append(g_dev)

# 7. (Optional) wrap in a DataLoader for batching
dev_loader = DataLoader(graphs_dev, batch_size=16, shuffle=True)

print(f"Built {len(graphs_dev)} graphs – example graph has {graphs_dev[0].num_nodes} nodes.")

graphs_test = []
for dlg_id, grp in df_test.groupby('dialogue_id', sort=True):
    grp = grp.sort_values('utterance_id')
    emb_list   = grp['emb_array'].tolist()
    sentiments = grp['sentiment_label'].tolist()
    emos_int   = [label2idx[e] for e in grp['label'].tolist()]
    g_test = build_linecongat_graph(emb_list, sentiments, emos_int)
    graphs_test.append(g_test)

# 7. (Optional) wrap in a DataLoader for batching
test_loader = DataLoader(graphs_test, batch_size=16, shuffle=True)

print(f"Built {len(graphs_test)} graphs – example graph has {graphs_test[0].num_nodes} nodes.")






In [ ]:
# pick one conversation to inspect, e.g. the very first
g = graphs_test[0]

# 1) Basic sizes
print("Num utterances (nodes):", g.num_nodes)
print("Num edges (including self-loops):", g.num_edges)
print("Node-feature shape (L, d):", g.x.shape)
print("Edge-attr shape (E, 2):", g.edge_attr.shape)
print("Labels shape (L,):", g.y.shape if hasattr(g, 'y') else None)
print()

# 2) Print the first few node embeddings (CLS vectors)
print("First 3 node embeddings:")
print(g.x[:3])
print()

# 3) Peek at the first 10 edges (src→dst) and their sentiment shifts
ei, ea = g.edge_index, g.edge_attr
print("First 10 edges (src→dst) with [sent_src, sent_dst]:")
for idx in range(min(10, ei.size(1))):
    print(f"  {ei[0,idx].item()} → {ei[1,idx].item()}  {ea[idx].tolist()}")
print()

# 4) Verify for each node i that it has edges to i−1, i, i+1
print("Neighbors and sentiment shifts for each node:")
for i in range(g.num_nodes):
    mask = (ei[0] == i)
    nbrs     = ei[1, mask].tolist()
    shifts   = ea[mask].tolist()
    print(f"  Node {i:2d}:  → {nbrs}  shifts={shifts}")

# 5) Finally, print the ground-truth labels for these utterances
print("\nEmotion labels (as ints):", g.y.tolist())


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATv2Conv

In [ ]:
# Cell 5: Model Definition
class GATv2Layer(nn.Module):
    def __init__(self, in_dim, out_dim, edge_dim=None, heads=6, concat=True,
                 negative_slope=0.2, dropout=0.0):
        super().__init__()
        self.conv = GATv2Conv(
            in_channels=in_dim, out_channels=out_dim,
            heads=heads, concat=concat,
            negative_slope=negative_slope, dropout=dropout,
            edge_dim=edge_dim
        )
        self.act = nn.ELU() if concat else nn.Identity()

    def forward(self, x, edge_index, edge_attr=None):
        out = self.conv(x, edge_index, edge_attr)
        return self.act(out)

class LineConGAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_classes,
                 edge_dim=2, heads=6, dropout=0.5):
        super().__init__()
        self.layer0 = GATv2Layer(in_dim, hidden_dim, edge_dim, heads, True, 0.2, dropout)
        self.layer1 = GATv2Layer(hidden_dim * heads, num_classes, edge_dim, heads, False, 0.2, dropout)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_attr,batch=None):
        h = self.layer0(x, edge_index, edge_attr)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        logits = self.layer1(h, edge_index, edge_attr)
        return logits

In [ ]:
# Cell 6: Instantiate Model & Optimizer
model = LineConGAT(
    in_dim=768, hidden_dim=128, num_classes=len(label2idx),
    edge_dim=2, heads=6, dropout=0.5
)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,        # learning rate 10^-3
    weight_decay=1e-4  # L2 regularization 10^-4
)
print(model)

In [ ]:
# Cell 7: Training Loop
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch
        optimizer.zero_grad()
        logits = model(batch.x, batch.edge_index, batch.edge_attr)
        loss = F.cross_entropy(logits, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}/{num_epochs} — Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
# Cell 8: Evaluation Function and Run
from sklearn.metrics import accuracy_score, f1_score

def evaluate(loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch
            logits = model(batch.x, batch.edge_index, batch.edge_attr)
            preds = logits.argmax(dim=1).cpu().tolist()
            labels = batch.y.cpu().tolist()
            all_preds.extend(preds); all_labels.extend(labels)
    return accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average='weighted')

train_acc, train_f1 = evaluate(train_loader)
dev_acc, dev_f1     = evaluate(dev_loader)
test_acc, test_f1   = evaluate(test_loader)
print(f"Train — Acc: {train_acc:.4f}, WA-F1: {train_f1:.4f}")
print(f"Dev   — Acc: {dev_acc:.4f}, WA-F1: {dev_f1:.4f}")
print(f"Test  — Acc: {test_acc:.4f}, WA-F1: {test_f1:.4f}")

In [ ]:
import torch
import matplotlib.pyplot as plt

# — Configuration —
graph_idx   = 0    # which dialogue-graph in train_graphs to inspect
target_node = 3    # which node (utterance index) to trace attention from

# 1) Select graph and move to device
data = graphs_train[graph_idx]

# 2) First-layer hidden reps
h = model.layer0(data.x, data.edge_index, data.edge_attr)
h = torch.relu(h)

# 3) Second-layer attention
_, (edge_index_clf, attn_weights) = model.layer1.conv(
    h, data.edge_index, data.edge_attr, return_attention_weights=True
)

# 4) Collapse heads → one score per edge
attn = attn_weights.mean(dim=1).cpu().detach()

# 5) Build per-node attention from target_node
N = data.num_nodes
node_attn = torch.zeros(N)
for eid, src in enumerate(edge_index_clf[0].cpu()):
    if src.item() == target_node:
        tgt = edge_index_clf[1, eid].item()
        node_attn[tgt] = attn[eid].item()

# 6) Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.bar(range(N), node_attn.numpy())
plt.xlabel("Utterance (node) index")
plt.ylabel(f"Attention weight from node {target_node}")
plt.title(f"Graph {graph_idx}: attention from node {target_node}")
plt.xticks(range(N))
plt.tight_layout()
plt.show()


In [ ]:
def plot_attention_heatmap_layer1(model, data):
    model.eval()
    data = data
    # Πρώτα φτιάχνουμε το h μετά τη layer0
    h = model.layer0(data.x, data.edge_index, data.edge_attr)
    h = F.relu(h)
    # Παίρνουμε attention weights από layer1
    _, (edge_idx, attn) = model.layer1.conv(
        h, data.edge_index, data.edge_attr, return_attention_weights=True
    )
    # Detach the tensor before converting to numpy
    attn = attn.mean(dim=1).cpu().detach().numpy()      # collapse heads
    edge_idx = edge_idx.cpu().numpy()
    N = data.num_nodes
    mat = np.zeros((N, N))
    for s, t, w in zip(edge_idx[0], edge_idx[1], attn):
        mat[s, t] = w
    plt.figure(figsize=(6,6))
    plt.imshow(mat, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title("Layer1 Attention Heatmap")
    plt.xlabel("Target node")
    plt.ylabel("Source node")
    plt.show()


In [ ]:
plot_attention_heatmap_layer1(model, graphs_train[0])


In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

class GuidedBackprop:
    def __init__(self, model):
        """
        Αντιγράψτε το μοντέλο και αντικαταστήστε κάθε ReLU με ένα
        όπου κατά το backward κρατάμε μόνο τα θετικά gradients.
        """
        self.model = model
        self.handles = []
        self._register_hooks()

    def _register_hooks(self):
        # Κάθε ReLU θα φιλτράρει τα αρνητικά gradients
        for module in self.model.modules():
            if isinstance(module, torch.nn.ReLU):
                h = module.register_backward_hook(self._relu_backward_hook)
                self.handles.append(h)

    def _relu_backward_hook(self, module, grad_in, grad_out):
        # Για κάθε είσοδο gradient grad_in[i], κρατάμε μόνο το θετικό μέρος:
        modified = []
        for g in grad_in:
            if g is None:
                modified.append(None)
            else:
                modified.append(F.relu(g))
        return tuple(modified)

    def generate_gradients(self, data, target_node, target_class):
        """
        Εκτελεί forward/backward και επιστρέφει το gradient του data.x
        Args:
          data         : ένα torch_geometric.data.Data graph
          target_node  : ο κόμβος που εξηγούμε (index)
          target_class : η κλάση (π.χ. argmax) για την οποία κάνουμε backprop
        Returns:
          node_grads   : Tensor [N, d] με τα guided gradients w.r.t. x
        """
        # 1) Σιγουρευόμαστε ότι το x έχει requires_grad
        data = data
        data.x = data.x.clone().detach().requires_grad_(True)

        # 2) Forward
        logits = self.model(data.x, data.edge_index, data.edge_attr)
        score  = logits[target_node, target_class]

        # 3) Backward guided
        self.model.zero_grad()
        score.backward(retain_graph=False)

        # 4) Πάρτε τα gradients w.r.t. x
        node_grads = data.x.grad.detach().cpu()  # [N, 768]
        return node_grads

    def clear_hooks(self):
        for h in self.handles:
            h.remove()
        self.handles = []

# —————— Παράδειγμα χρήσης ——————
# 1) Δημιουργούμε instance του GuidedBackprop
gb = GuidedBackprop(model)

# 2) Παίρνουμε ένα graph και βρίσκουμε το predicted class για έναν κόμβο
data = graphs_train[0]
with torch.no_grad():
    logits = model(data.x, data.edge_index, data.edge_attr)
preds = logits.argmax(dim=1).cpu()
target_node  = 5
target_class = preds[target_node].item()

# 3) Υπολογίζουμε τα guided gradients
node_grads = gb.generate_gradients(data, target_node, target_class)
# node_grads[i] είναι το gradient vector 768-διάστατο για κάθε κομβο i

# 4) Μετατρέπουμε σε node-importance π.χ. με L1-norm
import numpy as np
node_importance = node_grads.abs().sum(dim=1).numpy()  # [N]

# 5) Plot bar chart
plt.figure(figsize=(8,4))
plt.bar(range(data.num_nodes), node_importance)
plt.xlabel("Node (utterance) index")
plt.ylabel("Guided Backprop importance (L1 norm)")
plt.title(f"Guided Backprop for node {target_node}, class {target_class}")
plt.show()

# 6) Καθαρίζουμε hooks όταν τελειώσουμε
gb.clear_hooks()


In [ ]:

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def guided_backprop_heatmap(model, data):
    """
    Computes a heatmap H where H[i,j] is the guided backprop importance
    of node j for the prediction of node i.
    """
    model.eval()
    device = next(model.parameters()).device
    data = data.to(device)

    # Prepare guided backprop hooks
    gb = GuidedBackprop(model)

    # Get model predictions for each node
    with torch.no_grad():
        logits = model(data.x, data.edge_index, data.edge_attr)
    preds = logits.argmax(dim=1).cpu().numpy()

    N = data.num_nodes
    heat = np.zeros((N, N), dtype=float)

    # For each target node i
    for i in range(N):
        target_class = preds[i].item()
        # Compute guided gradients: shape [N, feature_dim]
        node_grads = gb.generate_gradients(data, i, target_class)
        # Aggregate per-source node j via L1 norm
        node_imp = node_grads.abs().sum(dim=1).numpy()  # [N]
        heat[i] = node_imp

    gb.clear_hooks()
    return heat

# Example usage:
data = graphs_train[0]  # first dialogue graph
heatmap = guided_backprop_heatmap(model, data)

# Plot heatmap
plt.figure(figsize=(8,6))
sns.heatmap(
    heatmap,
    xticklabels=range(data.num_nodes),
    yticklabels=range(data.num_nodes),
    cmap='viridis'
)
plt.xlabel("Contributing node j")
plt.ylabel("Target node i")
plt.title("Guided Backprop Importance Heatmap")
plt.show()


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.explain import Explainer, GNNExplainer

# 1) Select a conversation graph and move to device
data = graphs_train[0]  # first dialogue
# Ensure data has x, edge_index, edge_attr, y if available

# 2) Instantiate the Explainer for your LineConGAT model
explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config={
        'mode': 'multiclass_classification',
        'task_level': 'node',
        'return_type': 'log_probs',   # explains the log-softmax outputs
    },
)

# 3) Pick a node index to explain, e.g., the 6th utterance
target_node = 5

# 4) Generate the explanation
explanation = explainer(
    x=data.x,
    edge_index=data.edge_index,
    edge_attr=data.edge_attr,
    index=target_node
)

# 5) Access the learned masks
edge_importance = explanation.edge_mask    # Tensor of length E
node_importance = explanation.node_mask    # Tensor of length num_node_features

print("Edge mask:", edge_importance)
print("Node feature mask:", node_importance)


In [ ]:
import torch
from torch_geometric.explain import Explainer, GNNExplainer, ModelConfig

# 1) Pick a conversation‐graph and a target utterance‐node
data        = graphs_train[0]   # first dialogue
target_node = 5                            # utterance

# 2) Instantiate the new Explainer API with GNNExplainer
explainer = Explainer(
    model            = model,
    algorithm        = GNNExplainer(epochs=200),
    explanation_type = 'model',
    node_mask_type   = 'attributes',           # learn a mask over nodes
    edge_mask_type   = 'object',               # learn a mask over edges
    model_config     = ModelConfig(
                            mode='multiclass_classification',
                            task_level='node',
                            return_type='log_probs'
                       ),
)

# 3) Run the explainer for your target node
explanation = explainer(
    x     = data.x,
    edge_index = data.edge_index,
    edge_attr  = data.edge_attr,
    index = target_node
)

# 4) Print all outgoing edge‐importance from your target node
# 4) Print all edge‐importance scores for the entire graph
edge_idx  = data.edge_index.cpu().numpy()
edge_mask = explanation.edge_mask.cpu().numpy()
print("All edges and their learned importance:")
for src, tgt, w in zip(edge_idx[0], edge_idx[1], edge_mask):
    print(f"Node {src} → Node {tgt} : {w:.4f}")


explanation.visualize_graph(path="subgraph_visualization.png")
img = plt.imread("subgraph_visualization.png")
plt.imshow(img); plt.axis("off"); plt.show()

